<a href="https://colab.research.google.com/github/pjsandwich/bda-718-group-1/blob/master/Random_Forest_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark
pip install seaborn

In [3]:
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml import feature
import re
from pyspark.sql import functions as fn
from pyspark.sql.functions import col, regexp_replace, split
from pyspark.sql.utils import AnalysisException
from pyspark.sql import Row
from pyspark.sql.types import IntegerType, DateType, FloatType, BooleanType
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

spark = SparkSession \
  .builder \
  .master("local[*]")\
  .config("spark.memory.fraction", 0.8) \
  .config("spark.executor.memory", "12g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.memory.offHeap.enabled",'true')\
  .config("spark.memory.offHeap.size","12g")\
  .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql.functions import col, udf
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import create_map, lit
from itertools import chain
from pyspark.mllib import numpy as np
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.tuning import ParamGridBuilder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
new_df = spark.read.format("csv").option("header", "true").option('inferSchema','true').load("drive/Shared drives/IST 718 Group1/thecorrectstuff.csv")

In [5]:
reg_df = new_df
reg_df = reg_df.drop(*['latitude', 'listed_date', 'listing_color', 'longitude','city', 'dealer_zip','interior_color','sp_name','sp_id'])
reg_df = reg_df.withColumn('HasAccidents',fn.when(col('has_accidents') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('has_accidents')
 
reg_df = reg_df.withColumn('frameDamaged',fn.when(col('frame_damaged') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('frame_damaged')
 
reg_df = reg_df.withColumn('franchiseDealer',fn.when(col('franchise_dealer') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('franchise_dealer')
 
reg_df = reg_df.withColumn('isCabNew',fn.when(col('isCab') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('isCab')
 
reg_df = reg_df.withColumn('isCpo',fn.when(col('is_cpo') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('is_cpo')
 
#reg_df = reg_df.withColumn('isNew',fn.when(col('is_new') == 'True', 1).otherwise(0))
#reg_df = reg_df.drop('is_new')
 
reg_df = reg_df.withColumn('isOemcpo',fn.when(col('is_oemcpo') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('is_oemcpo')
 
reg_df = reg_df.withColumn('Salvage',fn.when(col('salvage') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('salvage')
 
reg_df = reg_df.withColumn('theftTitle',fn.when(col('theft_title') == 'True', 1).otherwise(0))
reg_df = reg_df.drop('theft_title')

reg_df = reg_df.withColumn('length_category',fn.when(col('length') < 176.4, "compact").when(col("length") > 187.2, "fullsize").otherwise('midsize'))
reg_df = reg_df.withColumn('width_category',fn.when(col('width') < 70.9, "narrow").when(col("width") > 74.8, "wide").otherwise("regular"))
reg_df = reg_df.withColumn('age',2020 - (col('year')))

from pyspark.ml import feature, Pipeline #import modules/packages for the entire process
# create ordinal pipline for select features
ord_cat_feat_eng_pipe = Pipeline(stages=[feature.StringIndexerModel.from_labels(['compact','midsize','fullsize'],
                                                                                handleInvalid = "keep", 
                                                                                inputCol="length_category",
                                                                                outputCol="length_category_idx"),
                                         feature.StringIndexerModel.from_labels(['narrow','regular','wide'],
                                                                                handleInvalid = "keep",
                                                                                inputCol="width_category",
                                                                                outputCol="width_category_idx")])

reg_df = ord_cat_feat_eng_pipe.fit(reg_df).transform(reg_df)

# Develop logic to scope out new cars 
#new_car_logic = (reg_df["isNew"] == 0)
#reg_df = reg_df[new_car_logic]

In [6]:
reg_df.show(5)

+------------+----+---------------+-----+-----------------+------------+----------------+-------------------+-----------+-----+--------------+-------------+----------------+---------+------+--------------------+----------+------+--------------------+-------------+---------------+-------+----------+-----------+-------+--------------+-------------+------------+--------------------+------------+---------+-----+----+-----------+----------+------------+-----------+-------------+-----------+--------------+--------------+------+-----------+------------+----------------+---------+----------------+------------+---------------+------+----+----+-----+---+------+---+---+---+----+----+----+----+----+-----+----+----+----+-------+-----+--------+----+--------+----+----+------+---------+----+----+-----+----+------+-----+------+-----------+----+----+----+----+----+------+----+------+----+-------+-----+-----+----+------------+------------+---------------+--------+-----+--------+----------+---------------

In [8]:
# skip this cell for now, too long
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

training_df, validation_df = reg_df.randomSplit([0.7, 0.3],
                                                seed=42)
# originals
#rf_cols = ['back_legroom','city_fuel_economy',
#           'daysonmarket','engine_cylinders','engine_displacement','fleet',
#           'frame_damaged','franchise_dealer','front_legroom',
#           'fuel_tank_volume','has_accidents','height',
#           'highway_fuel_economy','horsepower','isCab',
#           'is_cpo','is_oemcpo','length','maximum_seating','mileage',
#           'owner_count','salvage','savings_amount','seller_rating',
#           'theft_title','transmission_display','wheelbase','width','year',
#           'torque_ftlb','torque_rpm','torque_grade','n_body_type',
#           'n_engine_type','n_transmission','n_wheel_system','n_city',
#           'n_make_name','n_franchise_make','n_sp_name','n_interior_color',
#           'n_model_name','n_listing_color','n_fuel_type']

rf_cols = ["length_category_idx","width_category_idx","n_body_type",
           "n_engine_type","n_fuel_type","n_make_name","n_franchise_make",
           "n_model_name","n_transmission","n_wheel_system","age",
           "city_fuel_economy", "highway_fuel_economy","daysonmarket",
           "engine_cylinders", "horsepower","mileage","owner_count",
           "maximum_seating", "engine_displacement", "torque_ftlb"]

rf_vec = VectorAssembler(inputCols=rf_cols, outputCol='features')
rf = RandomForestRegressor(featuresCol='features', labelCol='price', maxBins=4482)

paramGrid = ParamGridBuilder().\
addGrid(rf.numTrees, np.arange(100,120,20)).\
addGrid(rf.maxDepth, np.arange(11,14,2)).\
build()

rf_pipe = Pipeline(stages = [rf_vec,rf])

all_models = []
for j in range(len(paramGrid)):
  print("Fitting model {}".format(j+1))
  model = rf_pipe.fit(training_df, paramGrid[j])
  all_models.append(model)
      
scores =[]
print('Model fitting complete -> Evaluating Scores')
for m in all_models:
  score_rf_pipe_1 = RegressionEvaluator(labelCol='price',metricName='rmse').evaluate(m.transform(validation_df))
  scores.append(score_rf_pipe_1)
        
print(min(scores), '\n', paramGrid[scores.index(min(scores))])

Fitting model 1


KeyboardInterrupt: ignored

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

training_df, validation_df = reg_df.randomSplit([0.7, 0.3],
                                                seed=42)

rf_cols = ["length_category_idx","width_category_idx","n_body_type",
           "n_engine_type","n_fuel_type","n_make_name","n_franchise_make",
           "n_model_name","n_transmission","n_wheel_system","age",
           "city_fuel_economy", "highway_fuel_economy","daysonmarket",
           "engine_cylinders", "horsepower","mileage","owner_count",
           "maximum_seating", "engine_displacement", "torque_ftlb"]

rf_vec = VectorAssembler(inputCols=rf_cols, outputCol='features')
rf = RandomForestRegressor(numTrees=100,
                           maxDepth=11,
                           featuresCol='features',
                           labelCol='price',
                           maxBins=4482)

rf_pipe = Pipeline(stages = [rf_vec,rf])

model = rf_pipe.fit(training_df)

score_rf_pipe_1 = RegressionEvaluator(labelCol='price',
                                      metricName='rmse').evaluate(model.transform(validation_df))

score_rf_pipe_1

3834.689402738222

In [10]:
model.stages[-1].featureImportances

SparseVector(21, {0: 0.0044, 1: 0.0149, 2: 0.0182, 3: 0.0071, 4: 0.0076, 5: 0.0228, 6: 0.0046, 7: 0.0213, 8: 0.0006, 9: 0.0331, 10: 0.1923, 11: 0.0081, 12: 0.0106, 13: 0.0018, 14: 0.0286, 15: 0.2917, 16: 0.1644, 17: 0.0205, 18: 0.0073, 19: 0.0315, 20: 0.1085})

In [11]:
predictions = model.transform(validation_df)

In [17]:
y_pred = [row[0] for row in predictions.select('prediction').collect()]
y_true = [row[0] for row in predictions.select('price').collect()]

In [18]:
import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {:4.3f}'.format(r2_score))

r2_score: 0.905


In [7]:
idx = np.argmin(scores)

all_models[idx].stages[-1].featureImportance

{Param(parent='RandomForestRegressor_23a1bf77cdd7', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 11,
 Param(parent='RandomForestRegressor_23a1bf77cdd7', name='numTrees', doc='Number of trees to train (>= 1).'): 100}

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



In [ ]:
new_df.describe().show()

+-------+------------------+-----+-----------+-----------+-----------+------------------+------------------+------------------+------------------+-------------------+-----------+--------------+------------------+------------------+---------+-----------------+--------------------+------------------+--------------------+------------------+------------------+-----------+-------------+------------------+--------------------+----------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+--------------------+------------+--------------------+------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+--------

In [ ]:
new_df.printSchema()

root
 |-- back_legroom: double (nullable = true)
 |-- bed: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: double (nullable = true)
 |-- daysonmarket: integer (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- engine_cylinders: integer (nullable = true)
 |-- engine_displacement: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- fleet: boolean (nullable = true)
 |-- frame_damaged: boolean (nullable = true)
 |-- franchise_dealer: boolean (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: double (nullable = true)
 |-- fuel_tank_volume: double (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- has_accidents: boolean (nullable = true)
 |-- height: double (nullable = true)
 |-- highway_fuel_economy: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- interior_color: string (n

In [ ]:
list(new_df.columns)

['back_legroom',
 'bed',
 'body_type',
 'cabin',
 'city',
 'city_fuel_economy',
 'daysonmarket',
 'dealer_zip',
 'engine_cylinders',
 'engine_displacement',
 'engine_type',
 'fleet',
 'frame_damaged',
 'franchise_dealer',
 'franchise_make',
 'front_legroom',
 'fuel_tank_volume',
 'fuel_type',
 'has_accidents',
 'height',
 'highway_fuel_economy',
 'horsepower',
 'interior_color',
 'isCab',
 'is_cpo',
 'is_new',
 'is_oemcpo',
 'latitude',
 'length',
 'listed_date',
 'listing_color',
 'longitude',
 'major_options',
 'make_name',
 'maximum_seating',
 'mileage',
 'model_name',
 'owner_count',
 'price',
 'salvage',
 'savings_amount',
 'seller_rating',
 'sp_id',
 'sp_name',
 'theft_title',
 'transmission',
 'transmission_display',
 'wheel_system',
 'wheelbase',
 'width',
 'year',
 'torque_ftlb',
 'torque_rpm',
 'torque_grade',
 'n_body_type',
 'n_engine_type',
 'n_fuel_type',
 'n_transmission',
 'n_wheel_system',
 'n_city',
 'n_make_name',
 'n_dealer_zip',
 'n_franchise_make',
 'n_sp_name',
 